# Examples of @map_e, @fmap_e and map_element
This notebook had examples of map from IoTPy/IoTPy/agent_types/op.py
You can create an agent that maps an input stream to an output stream using map_element, or the decorators @map_e or @fmap_e.
Note: The function map_element and the decorators @map_e and @fmap_e are essentially equivalent. Use the form that you find convenient.

In [1]:
import sys

In [2]:
sys.path.append('..')
sys.path.append("../IoTPy/helper_functions")
sys.path.append("../IoTPy/core")
sys.path.append("../IoTPy/agent_types")
from stream import Stream, run
from op import map_element
from recent_values import recent_values

## #Specify streams
x = Stream('x') 
specifies a stream x called 'x'.
### Specify terminating function that is wrapped to create non-terminating agent
def f(v): return 2 * v
takes a single input argument, returns a single value and terminates.
### Create a non-terminating agent that wraps f and reads stream x and extends stream y.
### y[n] = f(x[n]), all n
map_element(func=f, in_stream=x, out_stream=y)

In [3]:
def simple_example_of_map_element():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    # Specify encapsulated functions
    def f(v): return v+10
    # Create agent with input stream x and output stream y.
    map_element(func=f, in_stream=x, out_stream=y)
    # y[n] = x[n]+10

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [4]:
simple_example_of_map_element()

recent values of stream y are
[10, 11, 12, 13, 14]


## Example illustrating the decorator @map_e
The undecorated function f takes a single argument and returns a single value.
The decorated function f has two arguments, an in_stream and an out_stream, and may have additional keyword arguments.

In [5]:
from basics import map_e

def simple_example_of_map_e():
    # Specify streams
    x = Stream('x')
    y = Stream('y')

    # Decorate terminating function to specify non-terminating agent.
    @map_e
    def f(v): return v + 10
    # Create agent with input stream x and output stream y
    f(in_stream=x, out_stream=y)
    # y[n] = x[n]+10

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [6]:
simple_example_of_map_e()

recent values of stream y are
[10, 11, 12, 13, 14]


## Example illustrating the decorator @fmap_e
This is the functional form of @map_e
The output stream y doesn't have to be declared. f(x) creates and returns a stream.

In [7]:
from basics import fmap_e

def simple_example_of_fmap_e():
    # Specify streams
    x = Stream('x')

    # Decorate terminating function to specify non-terminating agent.
    @fmap_e
    def f(v): return v+10
    # Create agent with input stream x and output stream y
    y=f(x)
    # y[n] = x[n]+10

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [8]:
simple_example_of_fmap_e()

recent values of stream y are
[10, 11, 12, 13, 14]


## Building Networks of Agents by Connecting Output Streams to Input Streams
You can build networks of agents by connecting the output stream of agents to input streams of agents as shown in the next example. 

In [9]:
def example_of_concatentation_with_map_element():
    # Specify streams.
    x = Stream('x')
    y = Stream('y')
    w = Stream('w')

    # Specify encapsulated functions
    def f(v): return v+10
    def g(w): return w*2

    # Create agent with input stream x and output stream w.
    map_element(func=f, in_stream=x, out_stream=w)
    # y[n] = x[n]+10
    # Create agent with input stream w and output stream y
    map_element(func=g, in_stream=w, out_stream=y)

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [10]:
example_of_concatentation_with_map_element()

recent values of stream y are
[20, 22, 24, 26, 28]


## A Network of Agents is an Agent
You can use functions, in the usual way, to specify a function consisting of a network of agents. This function is itself a persistent agent: It reads its input streams and extends its output streams.

In the example below, we specify an agent h with two parameters, x and y, where x is an input stream and y is an output stream. Agent h is composed of two agents --- map_element(f, x, w) and map_element(g, w, y).

In [11]:
def example_of_network_of_agents_is_an_agent():

    # Specify an agent h with is a network of two agents
    # h has an input stream x, and an output stream y.
    def h(x, y):
        # Specify encapsulated functions local to h
        def f(v): return v+10
        def g(w): return w*2
        # Specify an internal stream of h
        w = Stream('w')
        # Specify agents local to h
        map_element(f, x, w)
        map_element(g, w, y)

    # Specify streams.
    x = Stream('x')
    y = Stream('y')

    # Create agent h which is a network of agents
    h(x, y)

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [12]:
example_of_network_of_agents_is_an_agent()

recent values of stream y are
[20, 22, 24, 26, 28]


## Function Composition of Agents
You can use fmap_e for a functional form, e.g., g(f(x)) in the exampe below.

In [13]:
def example_of_concatenating_fmap_e():
    # Specify streams
    x = Stream('x')

    # Decorate terminating function to specify non-terminating agent.
    @fmap_e
    def f(v): return v+10
    @fmap_e
    def g(w): return w * 2
    # Create agent with input stream x and output stream y
    y=g(f(x))
    # y[n] = (v+10)*2

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [14]:
example_of_concatenating_fmap_e()

recent values of stream y are
[20, 22, 24, 26, 28]


## Keyword arguments of an Agent
(Note: You can also use a class to store keyword arguments and the state.)
In the example below, the function add_constant has two parameters, v and ADDEND where v is an element of the input stream of the agent and ADDEND is a keyword parameter. The function returns a single value which is an element of the output stream of the agent. The call to map_element must have the keyword parameter, ADDEND.

In [15]:
def example_of_keyword_arg_with_map_element():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    
    # Specify encapsulated functions
    # This function operates on a variable v and has an
    # additional argument ADDEND which will be a keyword
    # argument to specify the agent.
    def add_constant(v, ADDEND): return v + ADDEND

    # Specify agents with keyword arguments
    map_element(func=add_constant, in_stream=x, out_stream=y,
               ADDEND=10)
    # y[n] = x[n] + 10

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [16]:
example_of_keyword_arg_with_map_element()

recent values of stream y are
[10, 11, 12, 13, 14]


In [17]:
def example_of_keyword_arg_with_map_e():
    # Specify streams
    x = Stream('x')
    y = Stream('y')

    # Decorate terminating function to specify non-terminating agent.
    @map_e
    def add_constant(v, ADDEND): return v + ADDEND
    # Create agent with input stream x and output stream y with keyword
    # argument
    add_constant(in_stream=x, out_stream=y, ADDEND=10)
    # y[n] = x[n] + 10

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [18]:
example_of_keyword_arg_with_map_e()

recent values of stream y are
[10, 11, 12, 13, 14]


## State of an Agent
The agent saves its state between successive calls to its wrapped function.

In the next example, the function f has two arguments, an input element and the state. The function may have additional keyword arguments. The function returns an output element and the next state. The initial state is specified in the call to map_element. In this example, the initial state is 0 because of the call map_element(func=f, in_stream=x, out_stream=y, state=0). Note that the call to map_element must have the keyword argument 'state'.

In [19]:
def example_of_state_with_map_element():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    # Specify encapsulated functions
    def f(input_element, state):
        next_output = input_element - state
        next_state = input_element
        return next_output, next_state
    # Create agents with input stream x and output stream y
    # and initial state of 0
    map_element(func=f, in_stream=x, out_stream=y, state=0)
    # state[0] = 0, state[n+1] = x[n]
    # y[0] = x[0], y[n+1] = x[n+1] - x[n]

    # Put test values in the input streams.
    x.extend([10, 20, 40, 80])
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [20]:
example_of_state_with_map_element()

recent values of stream y are
[10, 10, 20, 40]


In [21]:
def example_of_state_with_map_e():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    # Decorate encapsulated functions
    @map_e
    def f(input_element, state):
        next_output = input_element - state
        next_state = input_element
        return next_output, next_state
    # Create agents with input stream x and output stream y
    # and initial state of 0
    f(in_stream=x, out_stream=y, state=0)
    # state[0] = 0, state[n+1] = x[n] - state[n]
    # y[0] = x[0], y[n+1] = x[n+1] - x[n]

    # Put test values in the input streams.
    x.extend([10, 20, 40, 80])
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [22]:
example_of_state_with_map_e()

recent values of stream y are
[10, 10, 20, 40]


In [23]:
def example_of_state_with_concatenation_of_fmap_e():
    # Specify streams
    x = Stream('x')
    # Decorate encapsulated functions
    @fmap_e
    def f(input_element, state):
        next_output = input_element - state
        next_state = input_element
        return next_output, next_state
    @fmap_e
    def g(v): return v*2
    # Create agents with input stream x and output stream y
    # and initial state of 0
    y = g(f(x, state=0))
    # state[0] = 0, state[n+1] = x[n] - state[n]
    # y[0] = x[0], y[n+1] = x[n+1] - x[n]

    # Put test values in the input streams.
    x.extend([10, 20, 40, 80])
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [24]:
example_of_state_with_concatenation_of_fmap_e()

recent values of stream y are
[20, 20, 40, 80]


## Agents with both State and Keyword Arguments
The function that is encapsulated can have both state and additional keyword arguments. Note that the call to map_element must have keyword arguments 'state' and the additional keywords. In the following example the call to map_element specifies the initial state (state=0) and the value of the keyword argument (POWER=2).

In [25]:
def example_of_state_with_keyword_arg_with_map_element():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    # Specify encapsulated functions
    def f(input_element, state, POWER):
        next_output = input_element**POWER + state
        next_state = input_element + state
        return next_output, next_state
    # Create agents with input stream x and output stream y
    # and initial state of 0, and keyword arg POWER with value 2.
    map_element(func=f, in_stream=x, out_stream=y, state=0, POWER=2)
    # state[0] = 0, state[n+1] = x[0] + ... + x[n]
    # y[0] = x[0]**2,  y[n+1] = x[n+1]**2 + state[n]

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [26]:
example_of_state_with_keyword_arg_with_map_element()

recent values of stream y are
[0, 1, 5, 12, 22]


In [27]:
def example_of_state_with_keyword_arg_with_map_e():
    # Specify streams
    x = Stream('x')
    y = Stream('y')
    # Decorate encapsulated functions
    @map_e
    def f(input_element, state, POWER):
        next_output = input_element**POWER + state
        next_state = input_element + state
        return next_output, next_state
    # Create agents with input stream x and output stream y
    # and initial state of 0, and keyword arg POWER with value 2.
    f(in_stream=x, out_stream=y, state=0, POWER=2)
    # state[0] = 0, state[n+1] = x[0] + ... + x[n]
    # y[0] = x[0]**2,  y[n+1] = x[n+1]**2 + state[n]

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [28]:
example_of_state_with_keyword_arg_with_map_e()

recent values of stream y are
[0, 1, 5, 12, 22]


In [29]:
def example_of_state_with_keyword_arg_with_fmap_e():
    # Specify streams
    x = Stream('x')

    # Decorate encapsulated functions
    @fmap_e
    def f(input_element, state, POWER):
        next_output = input_element**POWER + state
        next_state = input_element + state
        return next_output, next_state
    # Create agents with input stream x and output stream y
    # and initial state of 0, and keyword arg POWER with value 2.
    y = f(x, state=0, POWER=2)
    # state[0] = 0, state[n+1] = x[0] + ... + x[n]
    # y[0] = x[0]**2,  y[n+1] = x[n+1]**2 + state[n]

    # Put test values in the input streams.
    x.extend(list(range(5)))
    # Execute a step
    run()
    # Look at recent values of streams.
    print ('recent values of stream y are')
    print (recent_values(y))

In [30]:
example_of_state_with_keyword_arg_with_fmap_e()

recent values of stream y are
[0, 1, 5, 12, 22]


## Saving the State of an Agent in an Argument of the Function
In the following example, the state of the agent is stored in a dict, s. The output of the example is the Fibonacci sequence. In this example, s[0] is the next output of the sequence and s[1] is the element following s[0].

In [31]:
def example_of_saving_state_in_argument():
    # y is the Fibonacci sequence
    x = Stream('x')
    y = Stream('y')
    # Save state in s
    s = {0:0, 1:1}

    def f(v, s):
        next = s[0]
        s[0], s[1] = s[1], s[0]+s[1]
        return next
    map_element(f, x, y, s=s)

    x.extend(list(range(10)))
    run()
    print ('recent values of stream y are')
    print (recent_values(y))

In [32]:
example_of_saving_state_in_argument()

recent values of stream y are
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


## Storing the State and Arguments of an Agent in a Class
The next example shows how you can save the state and arguments in a class. In this example, the state is running_sum which is the sum of the values read on the input stream, and multiplicand is an argument.

In [33]:
def example_class_to_save_state_and_args():
    class example(object):
        def __init__(self, multiplicand):
            self.multiplicand = multiplicand
            self.running_sum = 0
        def step(self, v):
            result = v * self.multiplicand + self.running_sum
            self.running_sum += v
            return result
    x = Stream()
    y = Stream()
    eg = example(multiplicand=2)
    map_element(func=eg.step, in_stream=x, out_stream=y)

    x.extend(list(range(5)))
    run()
    print ('recent values of stream y are')
    print (recent_values(y))
    

In [34]:
example_class_to_save_state_and_args()

recent values of stream y are
[0, 2, 5, 9, 14]
